In [1]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
from bokeh.layouts import column
from bokeh.models import TextInput, Button, Paragraph

def modify_doc(doc):
    
    # create some widgets
    button = Button(label="Say HI")
    input = TextInput(value="Bokeh")
    output = Paragraph()

    # add a callback to a widget
    def update():
        output.text = "Hello, " + input.value
    button.on_click(update)

    # create a layout for everything
    layout = column(button, input, output)

    # add the layout to curdoc
    doc.add_root(layout)
    
# In the notebook, just pass the function that defines the app to show
# You may need to supply notebook_url, e.g notebook_url="http://localhost:8889" 
show(modify_doc) 

In [10]:
from numpy.random import random

from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, TextInput

def get_data(N):
    return dict(x=random(size=N), y=random(size=N), r=random(size=N) * 0.03)

COLORS = ["black", "firebrick", "navy", "olive", "goldenrod"]

def modify_doc(doc):
    source = ColumnDataSource(data=get_data(200))

    p = figure(tools="", toolbar_location=None)
    r = p.circle(x='x', y='y', radius='r', source=source,
                 color="navy", alpha=0.6, line_color="white")

    
    select = Select(title="Color", value="navy", options=COLORS)
    input = TextInput(title="Number of points", value="200")

    def update_color(attrname, old, new):
        r.glyph.fill_color = select.value
    select.on_change('value', update_color)

    def update_points(attrname, old, new):
        N = int(input.value)
        source.data = get_data(N)
    input.on_change('value', update_points)

    layout = column(row(select, input, width=400), row(p))

    doc.add_root(layout)

show(modify_doc)

In [4]:
from math import cos, sin

from bokeh.models import ColumnDataSource

def modify_doc(doc):
    p = figure(match_aspect=True)
    p.circle(x=0, y=0, radius=1, fill_color=None, line_width=2)
    
    # this is just to help the auto-datarange
    p.rect(0, 0, 2, 2, alpha=0)

    # this is the data source we will stream to
    source = ColumnDataSource(data=dict(x=[1], y=[0]))
    p.circle(x='x', y='y', size=12, fill_color='white', source=source)

    def update():
        x, y = source.data['x'][-1], source.data['y'][-1]

        # construct the new values for all columns, and pass to stream
        new_data = dict(x=[x*cos(0.1) - y*sin(0.1)], y=[x*sin(0.1) + y*cos(0.1)])
        source.stream(new_data, rollover=2)

    doc.add_periodic_callback(update, 50)
    doc.add_root(p)
    
show(modify_doc)

In [5]:

channels = ['Main', 'Pool', 'Heater']
gauge_source = [1, 3., 2.]
p2 = figure(x_range=channels, plot_height=250, title='Current power')
p2.vbar(top=gauge_source,
        x=channels,
        width=0.9)
p2.yaxis.axis_label='Power (kW)'
show(p2)

In [11]:
import math
from bokeh.models import Range1d,Label
from bokeh.models import ColumnDataSource

p = figure(width=400, height=200, tools=[])

maxv = 10
def angle(x):
    x = min(x,maxv)
    return (1-x/maxv)*math.pi
value = 8.1
sa = angle(value)

ds = ColumnDataSource(dict(sa=[sa]))
p.annular_wedge(x=0, y=0, inner_radius=0.55, outer_radius=0.8,
                start_angle='sa', end_angle=math.pi, color="red", alpha=0.6, source = ds)

# cosmetics
p.y_range=Range1d(-0.1,1)
p.toolbar_location=None
p.axis.visible = False
p.grid.visible = False
mytext = Label(x=0, y=0, 
               text='{:3.2f}'.format(value), 
               text_font_size='50pt', text_align='center')
mytext2 = Label(x=0, y=-0.07, text='Main', text_font_size='16pt', text_align='center')

p.add_layout(mytext)
p.add_layout(mytext2)
    
ds.patch({'sa':[(0,angle(5))]})
show(p)


In [7]:
import pandas as pd
df = pd.DataFrame({'x':[0, 1], 'y':[2.,3]})
df

,x,y
0,0,2.0
1,1,3.0


In [8]:
df['y'].values

array([2., 3.])

In [9]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import Label, Button, CustomJS
from bokeh.layouts import gridplot

fig = figure()

fig.scatter([1,2],[1,2])

label = Label(x=1,y=1,text='Test')

fig.add_layout(label)

update_label = """
var now = new Date();
label.text = now.toString();
"""

button = Button(label='Change label',width=200,callback=CustomJS(args={'label':label},code=update_label))

show(gridplot([[fig,button]],toolbar_location=None))